In [1]:
import pandas as pd
import re
import plotly.express as px
from scipy.optimize import brentq
from datetime import date, timedelta
from rich.pretty import pprint
import math

In [2]:
def init_cashflow(drawdown, n, start_date):
  d = start_date.day
  start_date -= timedelta(days=d)
  ix = pd.date_range(start=start_date, periods=n+1 , freq='MS')
  ix = ix + timedelta(days=d-1)
  days = ix.to_series().diff().dt.days.fillna(0).astype(int)
  df = pd.DataFrame({'days' : days, 'capital' : float(drawdown), 'repayment' : 0., 'interest': 0., 'int_paid':0., 'frac_int':0., 'outstanding':0.}, index=ix)
  df.iat[0,6] = df.iat[0,1]
  return df

def evaluate_cashflow(df, repayment, interest_rate):
  '''updates df and return the final outstanding amount given a fixed repayment amount'''
  df.iloc[1:,2] = repayment
  prev = None
  for i in df.index:
    # copy prev outstanding to current capital
    if prev:
      df.at[i,'capital'] = df.at[prev,'outstanding']
    # calculate row
    carried_interest = df.at[prev,'frac_int'] if prev else 0.
    df.at[i,'interest'] = df.at[i,'days'] * df.at[i,'capital'] * interest_rate / 365 + carried_interest
    df.at[i,'int_paid'] = math.floor(df.at[i,'interest'] * 100) / 100
    df.at[i,'frac_int'] = df.at[i,'interest'] - df.at[i,'int_paid']
    df.at[i,'outstanding'] = df.at[i,'capital'] - df.at[i,'repayment'] + df.at[i,'int_paid']
    prev = i
  return df.iat[-1,6]

def solve_cashflow(df, interest_rate):
  ubound = df.iat[0,1]
  return brentq(lambda x: evaluate_cashflow(df, x, interest_rate), 0, ubound)

def example_cashflow(drawdown: float, interest_rate: float, months: int):
    df = init_cashflow(drawdown, months, date.today())
    r = solve_cashflow(df, interest_rate)
    # re-evaluate with rounded repayment
    evaluate_cashflow(df, round(r,2), interest_rate)
    print('--- example cashflow ---')
    print(f'drawdown {drawdown:.2f}, interest rate {interest_rate * 100:.4f}%, monthly payment {round(r,2):.2f}')
    print(f'total paid {df["repayment"].sum():.2f} , total interest {df["int_paid"].sum():.2f}, total frac {df["frac_int"].sum()}')
    
    return df

In [3]:
example_cashflow(1200, 0.06, 12)

--- example cashflow ---
drawdown 1200.00, interest rate 6.0000%, monthly payment 103.27
total paid 1239.24 , total interest 39.26, total frac 0.052878356164431595


,days,capital,repayment,interest,int_paid,frac_int,outstanding
2024-09-19,0,1200.00,0.00,0.000000,0.00,0.000000,1200.00
2024-10-19,30,1200.00,103.27,5.917808,5.91,0.007808,1102.64
2024-11-19,31,1102.64,103.27,5.626741,5.62,0.006741,1004.99
2024-12-19,30,1004.99,103.27,4.962856,4.96,0.002856,906.68
2025-01-19,31,906.68,103.27,4.623198,4.62,0.003198,808.03
2025-02-19,31,808.03,103.27,4.120830,4.12,0.000830,708.88
2025-03-19,28,708.88,103.27,3.263620,3.26,0.003620,608.87
2025-04-19,31,608.87,103.27,3.106355,3.10,0.006355,508.70
2025-05-19,30,508.70,103.27,2.515013,2.51,0.005013,407.94
2025-06-19,31,407.94,103.27,2.083830,2.08,0.003830,306.75


In [14]:
def check_seq(num1s: int):
    """generator method. create an fsm that looks for 1s followed by 2s then a 1"""
    while True:
        xs = []
        y = yield False
        found = 0
        if y == 1:
            while y == 1:
                found += 1
                xs.append(y)
                y = yield False
            if y == 2 and found >= num1s:
                while y == 2:
                    xs.append(y)
                    y = yield False
                if y == 1:
                    pprint(xs)
                    y = yield True

In [17]:
fn = check_seq(3)
next(fn) # init the generator
xs = [fn.send(e) for e in [0,1,2,1,1,1,2,2,1,0,1]]
pprint(xs)

[1, 1, 1, 2, 2]

[False, False, False, False, False, False, False, False, True, False, False]

In [8]:
s = 'Certainly! To compare \\( 2^{\\frac{1}{2}} \\) and \\( 3^{\\frac{1}{3}} \\) algebraically, we can raise both sides to the power of 6 to eliminate the roots. This is because the least common multiple of the denominators (2 and 3) is 6.\n\nStarting with the comparison:\n\n\\[\n2^{\\frac{1}{2}} \\text{ vs. } 3^{\\frac{1}{3}}\n\\]\n\nWe will raise both sides to the power of 6:\n\n\\[\n(2^{\\frac{1}{2}})^6 \\text{ vs. } (3^{\\frac{1}{3}})^6\n\\]\n\nThis simplifies to:\n\n\\[\n2^3 \\text{ vs. } 3^2\n\\]\n\nCalculating both sides:\n\n\\[\n2^3 = 8\n\\]\n\\[\n3^2 = 9\n\\]\n\nNow we can compare the two results:\n\n\\[\n8 < 9\n\\]\n\nThus, we conclude that:\n\n\\[\n2^{\\frac{1}{2}} < 3^{\\frac{1}{3}}\n\\]\n\nSo, \\( 3^{\\frac{1}{3}} \\) is indeed greater than \\( 2^{\\frac{1}{2}} \\).'
xs = s.splitlines()
pprint(xs)

[
│   'Certainly! To compare \\( 2^{\\frac{1}{2}} \\) and \\( 3^{\\frac{1}{3}} \\) algebraically, we can raise both sides to the power of 6 to eliminate the roots. This is because the least common multiple of the denominators (2 and 3) is 6.',
│   '',
│   'Starting with the comparison:',
│   '',
│   '\\[',
│   '2^{\\frac{1}{2}} \\text{ vs. } 3^{\\frac{1}{3}}',
│   '\\]',
│   '',
│   'We will raise both sides to the power of 6:',
│   '',
│   '\\[',
│   '(2^{\\frac{1}{2}})^6 \\text{ vs. } (3^{\\frac{1}{3}})^6',
│   '\\]',
│   '',
│   'This simplifies to:',
│   '',
│   '\\[',
│   '2^3 \\text{ vs. } 3^2',
│   '\\]',
│   '',
│   'Calculating both sides:',
│   '',
│   '\\[',
│   '2^3 = 8',
│   '\\]',
│   '\\[',
│   '3^2 = 9',
│   '\\]',
│   '',
│   'Now we can compare the two results:',
│   '',
│   '\\[',
│   '8 < 9',
│   '\\]',
│   '',
│   'Thus, we conclude that:',
│   '',
│   '\\[',
│   '2^{\\frac{1}{2}} < 3^{\\frac{1}{3}}',
│   '\\]',
│   '',
│   'So, \\( 3^{\\frac{1}{3}} \\) is indeed greater than \\( 2^{\\frac{1}{2}} \\).'
]

In [18]:
ys = []
for s in xs:
    if s == '\\[' or s == '\\]':
        s = '$$'
    x = re.sub(r'\\\(\s*(.*?)\s*\\\)', r'$$ \1 $$', s)
    ys.append(x)
#    pprint(x)
y = '\n'.join(ys)
pprint(y)
#    y = re.sub(r'\\\\\[\\n*(.*?)\\n\\\\\]', r'$$ \1 $$', x)
    # pprint(re.sub(r'\[\s*(.*?)\s*\]', r'$$ \1 $$', s))

'Certainly! To compare $$ 2^{\\frac{1}{2}} $$ and $$ 3^{\\frac{1}{3}} $$ algebraically, we can raise both sides to the power of 6 to eliminate the roots. This is because the least common multiple of the denominators (2 and 3) is 6.\n\nStarting with the comparison:\n\n$$\n2^{\\frac{1}{2}} \\text{ vs. } 3^{\\frac{1}{3}}\n$$\n\nWe will raise both sides to the power of 6:\n\n$$\n(2^{\\frac{1}{2}})^6 \\text{ vs. } (3^{\\frac{1}{3}})^6\n$$\n\nThis simplifies to:\n\n$$\n2^3 \\text{ vs. } 3^2\n$$\n\nCalculating both sides:\n\n$$\n2^3 = 8\n$$\n$$\n3^2 = 9\n$$\n\nNow we can compare the two results:\n\n$$\n8 < 9\n$$\n\nThus, we conclude that:\n\n$$\n2^{\\frac{1}{2}} < 3^{\\frac{1}{3}}\n$$\n\nSo, $$ 3^{\\frac{1}{3}} $$ is indeed greater than $$ 2^{\\frac{1}{2}} $$.'

In [21]:
(4*6*1) % 7

3